In [41]:
import torch
from transformers import CLIPModel, BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline, ViTFeatureExtractor

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# model = ViTFeatureExtractor.from_pretrained("openai/clip-vit-large-patch14")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall").to(device)
# model

In [42]:
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

In [33]:
def generate(text, model, tokenizer, max_length):

    def generate_loop(data):
        with torch.no_grad():
            out = model(**data)
        
        out = out['logits'][:, -1]

        topk_value = torch.topk(out, 50).values
        topk_value = topk_value[:, -1].unsqueeze(dim=1)

        # 赋值
        out = out.masked_fill(out < topk_value, -float('inf'))  # 类似与torch.where

        # 根据概率采样, 无放回, 所以不可能重复
        out = out.softmax(dim=1)
        out = out.multinomial(num_samples=1)    # 值就是采样权重(概率), replacement默认为False不放回, 返回该值的下标

        data['input_ids'] = torch.cat([data['input_ids'], out], dim=1)
        data['attention_mask'] = torch.ones_like(data['input_ids'])

        if(data['input_ids'].shape[1] >= max_length):   # 大于max_length
            return data
        
        return generate_loop(data)

    # 重复5遍
    data = tokenizer.batch_encode_plus([text]*5, return_tensors='pt', return_token_type_ids=False, add_special_tokens=False)
    for k in data.keys():
        data[k] = data[k].to(device)
        
    data = generate_loop(data)

    for i in range(5):
        print(i, tokenizer.decode(data['input_ids'][i]))

In [44]:
# generate('这是很久之前的事情了', model, tokenizer, 100)
data = tokenizer(['text'], return_tensors='pt', return_token_type_ids=False, return_overflowing_tokens=False, add_special_tokens=False)
data

{'input_ids': tensor([[10539]]), 'attention_mask': tensor([[1]])}